In [ ]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

In [ ]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

tf.keras.utils.set_random_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [ ]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [ ]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
#build model

def define_model(seedval):
    inputs = tf.keras.Input(shape=(72, 1), name='input')
    x = Conv1D(filters=15, kernel_size=3,strides =3)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Conv1D(filters=30, kernel_size=5,strides =2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dropout(0.5,seed=seedval)(x)
    x = Dense(20,activation ='relu')(x)
    x = Dense(10,activation ='relu')(x)
    x = Dense(5,activation ='relu')(x)
    outputs = Dense(3, activation='softmax', name='predictions')(x)
    cnn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics=['accuracy'])
    return cnn_model

In [ ]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=100)
checkpoint_path = 'alarglysclassweights_march3spring24.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [ ]:
model = define_model(seed_value)
print(model.summary())

In [ ]:

history = model.fit(X_train, y_train,
              batch_size=5,
              verbose=1,
              epochs=400,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                    ,class_weight = {0:2.4,1:2.23,2:7.37}
                   )

In [ ]:
model1 = tf.keras.models.load_model('alarglysclassweights_march3spring24.h5')
model1.evaluate(X_test,y_test)

In [ ]:
y=model1.predict(X_test)

In [ ]:
#class-wise accuracy
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.to_numpy().argmax(axis=1), y.argmax(axis=1))
matrix.diagonal()/matrix.sum(axis=1)